# Redukcja wymiarów

In [ ]:
# import bibliotek
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import numpy as np

from sklearn import preprocessing
from sklearn import decomposition
from sklearn.feature_selection import VarianceThreshold

Wczytanie danych dotyczących oceny jakości portugalskiego białego wina `Vinho Verde` na podstawie danych z próbek i sensorów. Eksperci oceniali wino w skali od 0 do 10 (bardzo złe i bardzo dobre odpowiednio). [Więcej o danych...](https://bit.ly/2FiwKmy)

In [ ]:
# wczytanie danych
wino_df = pd.read_csv("https://bit.ly/2XYWtqG")
wino_df.head()

,Unnamed: 0,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,1,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,2,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,3,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,5,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [ ]:
wino_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            4898 non-null   int64  
 1   fixed.acidity         4898 non-null   float64
 2   volatile.acidity      4898 non-null   float64
 3   citric.acid           4898 non-null   float64
 4   residual.sugar        4898 non-null   float64
 5   chlorides             4898 non-null   float64
 6   free.sulfur.dioxide   4898 non-null   float64
 7   total.sulfur.dioxide  4898 non-null   float64
 8   density               4898 non-null   float64
 9   pH                    4898 non-null   float64
 10  sulphates             4898 non-null   float64
 11  alcohol               4898 non-null   float64
 12  quality               4898 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 497.6 KB


In [ ]:
# wizualizacja zależności między kolumnami
kolumny = pd.Index(wino_df.columns).tolist()
wykres = px.scatter_matrix(wino_df, dimensions=kolumny[:-1], color="quality", hover_name = "Unnamed: 0") #dodaję hover_name aby móc łatwo znaleźć outliers (po indeksie)
wykres.update_layout(autosize=False, width=1600, height=1600)
wykres.show()

In [ ]:
# lista cech objaśniających
kolumny[:-1]

['Unnamed: 0',
 'fixed.acidity',
 'volatile.acidity',
 'citric.acid',
 'residual.sugar',
 'chlorides',
 'free.sulfur.dioxide',
 'total.sulfur.dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [ ]:
# utworzenie zbiru cech objasniających i zmiennej objaśnianej
X = wino_df[kolumny[:-1]].drop(columns='Unnamed: 0')
y = wino_df["quality"]

## Za pomocą wyboru cech

### Ograniczenie względem wariancji

Wzór na wyliczenie wariancji:<br>
<img src="https://akademiadatascience.s3-eu-west-1.amazonaws.com/materialy/sds/modul6/wariancja.jpg" width="250">

In [ ]:
# sprawdzenie wariancji
np.var(X)

fixed.acidity              0.711968
volatile.acidity           0.010157
citric.acid                0.014643
residual.sugar            25.720518
chlorides                  0.000477
free.sulfur.dioxide      289.183667
total.sulfur.dioxide    1805.716751
density                    0.000009
pH                         0.022797
sulphates                  0.013022
alcohol                    1.514118
dtype: float64

In [ ]:
# zdefiniowanie wartości progowej
prog = VarianceThreshold(threshold=.01)
# trenowanie funkcji
funkcja = prog.fit(X)
# tworzenie macierzy cech o wysokiej zmienności
cechy_silnie_zmienne = funkcja.transform(X)
# Wyświetlenie macierzy cech o wysokiej wariancji.
cechy_silnie_zmienne

array([[ 7.  ,  0.27,  0.36, ...,  3.  ,  0.45,  8.8 ],
       [ 6.3 ,  0.3 ,  0.34, ...,  3.3 ,  0.49,  9.5 ],
       [ 8.1 ,  0.28,  0.4 , ...,  3.26,  0.44, 10.1 ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  2.99,  0.46,  9.4 ],
       [ 5.5 ,  0.29,  0.3 , ...,  3.34,  0.38, 12.8 ],
       [ 6.  ,  0.21,  0.38, ...,  3.26,  0.32, 11.8 ]])

In [ ]:
X.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9


In [ ]:
funkcja.variances_

array([7.11968197e-01, 1.01574668e-02, 1.46428029e-02, 2.57205179e+01,
       4.77236255e-04, 2.89183667e+02, 1.80571675e+03, 8.94369782e-06,
       2.27965259e-02, 1.30220468e-02, 1.51411779e+00])

In [ ]:
X.head()

### Ograniczenie wzgledem korelacji

Wzór na wyliczenie korelacji miédzy dwoma zmiennymi:<br>
<img src="https://akademiadatascience.s3-eu-west-1.amazonaws.com/materialy/sds/modul6/korelacja.png" width="300">

In [ ]:
print(X)
print(y)

      fixed.acidity  volatile.acidity  citric.acid  ...    pH  sulphates  alcohol
0               7.0              0.27         0.36  ...  3.00       0.45      8.8
1               6.3              0.30         0.34  ...  3.30       0.49      9.5
2               8.1              0.28         0.40  ...  3.26       0.44     10.1
3               7.2              0.23         0.32  ...  3.19       0.40      9.9
4               7.2              0.23         0.32  ...  3.19       0.40      9.9
...             ...               ...          ...  ...   ...        ...      ...
4893            6.2              0.21         0.29  ...  3.27       0.50     11.2
4894            6.6              0.32         0.36  ...  3.15       0.46      9.6
4895            6.5              0.24         0.19  ...  2.99       0.46      9.4
4896            5.5              0.29         0.30  ...  3.34       0.38     12.8
4897            6.0              0.21         0.38  ...  3.26       0.32     11.8

[4898 rows x 11

Wskazówka: funkcja `abs()` służy do zwracania wartości bezwzględnej liczby.

In [ ]:
# sprawdzenie korelacji
korelacja = X.corr().abs()
print(korelacja)
print(korelacja)

                      fixed.acidity  volatile.acidity  ...  sulphates   alcohol
fixed.acidity              1.000000          0.022697  ...   0.017143  0.120881
volatile.acidity           0.022697          1.000000  ...   0.035728  0.067718
citric.acid                0.289181          0.149472  ...   0.062331  0.075729
residual.sugar             0.089021          0.064286  ...   0.026664  0.450631
chlorides                  0.023086          0.070512  ...   0.016763  0.360189
free.sulfur.dioxide        0.049396          0.097012  ...   0.059217  0.250104
total.sulfur.dioxide       0.091070          0.089261  ...   0.134562  0.448892
density                    0.265331          0.027114  ...   0.074493  0.780138
pH                         0.425858          0.031915  ...   0.155951  0.121432
sulphates                  0.017143          0.035728  ...   1.000000  0.017433
alcohol                    0.120881          0.067718  ...   0.017433  1.000000

[11 rows x 11 columns]
                

In [ ]:
# zdefiniowanie macierzy do filtrowania
macierz_filter = np.triu(np.ones(korelacja.shape), k=1).astype(np.bool)
print(macierz_filter)

[[False  True  True  True  True  True  True  True  True  True  True]
 [False False  True  True  True  True  True  True  True  True  True]
 [False False False  True  True  True  True  True  True  True  True]
 [False False False False  True  True  True  True  True  True  True]
 [False False False False False  True  True  True  True  True  True]
 [False False False False False False  True  True  True  True  True]
 [False False False False False False False  True  True  True  True]
 [False False False False False False False False  True  True  True]
 [False False False False False False False False False  True  True]
 [False False False False False False False False False False  True]
 [False False False False False False False False False False False]]


In [ ]:
# wybranie górnej części trójkąta w macierzy korelacji
gorny_trojkat = korelacja.where(macierz_filter)
print(gorny_trojkat)

                      fixed.acidity  volatile.acidity  ...  sulphates   alcohol
fixed.acidity                   NaN          0.022697  ...   0.017143  0.120881
volatile.acidity                NaN               NaN  ...   0.035728  0.067718
citric.acid                     NaN               NaN  ...   0.062331  0.075729
residual.sugar                  NaN               NaN  ...   0.026664  0.450631
chlorides                       NaN               NaN  ...   0.016763  0.360189
free.sulfur.dioxide             NaN               NaN  ...   0.059217  0.250104
total.sulfur.dioxide            NaN               NaN  ...   0.134562  0.448892
density                         NaN               NaN  ...   0.074493  0.780138
pH                              NaN               NaN  ...   0.155951  0.121432
sulphates                       NaN               NaN  ...        NaN  0.017433
alcohol                         NaN               NaN  ...        NaN       NaN

[11 rows x 11 columns]


In [ ]:
gorny_trojkat.max()

fixed.acidity                NaN
volatile.acidity        0.022697
citric.acid             0.289181
residual.sugar          0.094212
chlorides               0.114364
free.sulfur.dioxide     0.299098
total.sulfur.dioxide    0.615501
density                 0.838966
pH                      0.425858
sulphates               0.155951
alcohol                 0.780138
dtype: float64

In [ ]:
# zdefiniowanie indeksów kolumn o korelacji powyżej 0.9
indeks_mocna_korelacja = [kolumna for kolumna in gorny_trojkat.columns if any(gorny_trojkat[kolumna] > 0.8)]
print(indeks_mocna_korelacja)

['density']


In [ ]:
# usunięcie silnie skorelowanych cech
X.drop(columns=indeks_mocna_korelacja).head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,3.19,0.40,9.9


## PCA

Zobrazowanie różnego rodzaju wymiarów od 1D do 5D:<br>
![wymiary](https://upload.wikimedia.org/wikipedia/commons/5/5e/Dice_analogy-_1_to_5_dimensions.svg)

In [ ]:
print(X)
print(y)

      fixed.acidity  volatile.acidity  citric.acid  ...    pH  sulphates  alcohol
0               7.0              0.27         0.36  ...  3.00       0.45      8.8
1               6.3              0.30         0.34  ...  3.30       0.49      9.5
2               8.1              0.28         0.40  ...  3.26       0.44     10.1
3               7.2              0.23         0.32  ...  3.19       0.40      9.9
4               7.2              0.23         0.32  ...  3.19       0.40      9.9
...             ...               ...          ...  ...   ...        ...      ...
4893            6.2              0.21         0.29  ...  3.27       0.50     11.2
4894            6.6              0.32         0.36  ...  3.15       0.46      9.6
4895            6.5              0.24         0.19  ...  2.99       0.46      9.4
4896            5.5              0.29         0.30  ...  3.34       0.38     12.8
4897            6.0              0.21         0.38  ...  3.26       0.32     11.8

[4898 rows x 11

### Scikit-Learn: PCA()

#### Wybór liczby wymiarów

Dobór liczby wymiarów na podstawie wybrania poziomu wariancji zmiennych, który chcemy zachować w zbiorze danych.

In [ ]:
# tworzenie kompresora
kompresor_pca = decomposition.PCA(n_components=0.95)
# wyuczenie kompresora
fit_pca = kompresor_pca.fit(X)
# użycie kompresora
X_VAR = fit_pca.transform(X)
print(X_VAR)

[[ 33.7327539    1.23828497]
 [-11.92579973 -18.95751887]
 [-41.22467147   5.73987365]
 ...
 [-27.99422284   1.98656157]
 [-31.62562278  -7.36798453]
 [-42.66909003  -2.30113142]]


#### Redukowanie wymiarowości do podanej liczby zmiennych


##### 2D

In [ ]:
## Redukcja zbioru do 2 wymiarów
# tworzenie kompresora
kompresor_pca = decomposition.PCA(n_components=2)
# wyuczenie kompresora
fit_pca = kompresor_pca.fit(X)
# użycie kompresora
X_2D = fit_pca.transform(X)
print(X_2D)

[[ 33.7327539    1.23828497]
 [-11.92579973 -18.95751887]
 [-41.22467147   5.73987365]
 ...
 [-27.99422284   1.98656157]
 [-31.62562278  -7.36798453]
 [-42.66909003  -2.30113142]]


In [ ]:
# tworzenie zbioru dla wykresu
X_2D_df = pd.DataFrame(X_2D, columns=["x1", "x2"])
X_2D_df["ocena"] = y
# wykres dla 2 wymiarów
wykres = px.scatter(X_2D_df, x="x1", y="x2", color="ocena", log_x=True)
wykres.show()

##### 3D

In [ ]:
## Redukcja zbioru do 3 wymiarów
# tworzenie kompresora
kompresor_pca = decomposition.PCA(n_components=3)
# wyuczenie kompresora
fit_pca = kompresor_pca.fit(X)
# użycie kompresora
X_3D = fit_pca.transform(X)
print(X_3D)

[[ 33.7327539    1.23828497  12.77529181]
 [-11.92579973 -18.95751887  -3.85783398]
 [-41.22467147   5.73987365   2.4660316 ]
 ...
 [-27.99422284   1.98656157  -3.76679051]
 [-31.62562278  -7.36798453  -3.85584946]
 [-42.66909003  -2.30113142  -3.60383843]]


In [ ]:
# wykres dla 3 wymiarów - 1
X_3D_df = pd.DataFrame(X_3D, columns=["x1", "x2", "x3"])
X_3D_df["ocena"] = y
wykres = px.scatter_3d(X_3D_df,
                       x="x1",
                       y="x2",
                       z="x3",
                       color="ocena")
wykres.show()

In [ ]:
# wykres dla 3 wymiarów - 2
wykres = go.Figure(data=[go.Surface(z=X_3D)])
wykres.update_layout(title="Zbiór danych zredukowany do 3 wymiarów")

### Scit-Learn: KernelPCA()

Jądrowa analiza PCA, gdzie można wybrać dowolny `kernel` z listy:
* `linear` - jądro liniowe;
* `poly` -  jądro wielomianowe;
* `rbf` - jądro radialnej funkcji bazowej;
* `cosine` - jądro kosinusowe.

##### 2D

In [ ]:
## Redukcja zbioru do 2 wymiarów
# tworzenie kompresora
kompresor_pca_kernel = decomposition.KernelPCA(n_components=2, kernel="rbf")
# wyuczenie kompresora
fit_pca_kernel = kompresor_pca_kernel.fit(X)
# użycie kompresora
X_2D_kernel = fit_pca_kernel.transform(X)
print(X_2D_kernel)

[[-0.006959   -0.00705223]
 [-0.01104746 -0.01169776]
 [-0.02298976  0.09207883]
 ...
 [ 0.25223008 -0.01905768]
 [ 0.20415716  0.00464675]
 [-0.03342393  0.15642582]]


In [ ]:
# tworzenie zbioru dla wykresu
X_2D_kernel_df = pd.DataFrame(X_2D_kernel, columns=["x1", "x2"])
X_2D_kernel_df["ocena"] = y
# wykres dla 2 wymiarów
wykres = px.scatter(X_2D_kernel_df, x="x1", y="x2", color="ocena", log_x=True)
wykres.show()

##### 3D

In [ ]:
## Redukcja zbioru do 3 wymiarów
# tworzenie kompresora
kompresor_pca_kernel = decomposition.KernelPCA(n_components=3, kernel="rbf")
# wyuczenie kompresora
fit_pca_kernel = kompresor_pca_kernel.fit(X)
# użycie kompresora
X_3D_kernel = fit_pca_kernel.transform(X)
print(X_3D_kernel)

[[-0.006959   -0.00705223 -0.00401555]
 [-0.01104746 -0.01169776 -0.00890086]
 [-0.02298976  0.09207883  0.00615953]
 ...
 [ 0.25223008 -0.01905768  0.2048125 ]
 [ 0.20415716  0.00464675 -0.11041185]
 [-0.03342393  0.15642582 -0.08684003]]


In [ ]:
# wykres dla 3 wymiarów - 1
X_3D_kernel_df = pd.DataFrame(X_3D_kernel, columns=["x1", "x2", "x3"])
X_3D_kernel_df["ocena"] = y
wykres = px.scatter_3d(X_3D_kernel_df,
                       x="x1",
                       y="x2",
                       z="x3",
                       color="ocena")
wykres.show()

In [ ]:
# wykres dla 3 wymiarów - 2
wykres = go.Figure(data=[go.Surface(z=X_3D_kernel)])
wykres.update_layout(title="Zbiór danych zredukowany do 3 wymiarów")